<a href="https://colab.research.google.com/github/agarwalsourabh55/huggingface-tapas/blob/main/huggingface_tapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers torch-scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 44.8 MB/s 
     |████████████████████████████████| 120 kB 49.1 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=54012ebe5cbdabf6a1010d56ac13db69debf0c59fcf75b6838e5065e968ff617
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


### Different Types of Declaring TAPAS models


1.   Default SQA Configutaion
2.   with WTQ Configuration
3.   With WikiSQL Configuration


In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering

# for example, the base sized model with default SQA configuration
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")

# or, the base sized model with WTQ configuration
config = TapasConfig.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# # or, the base sized model with WikiSQL configuration
# config = TapasConfig( "google/tapas-base-finetuned-wikisql-supervised",
#                         num_aggregation_labels=3,
#                         average_logits_per_cell=True)
# model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'column_output_bias', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'aggregation_classifier.weight', 'output_bias', 'aggregation_classifier.bias', 'column_output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



### Defining the Tokenizer class


1.   Defining the data labels.
2.   Defining the querie.
3.   Defining the answers of that queries.
4.   Converting all into the DAtaframe if not.
5.   Tokenizer Takes Text Only. 
6.   Advised to use Dataloaders for training the TAPAS



In [ ]:
from transformers import TapasTokenizer
import pandas as pd

model_name = "google/tapas-base"
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
answer_coordinates = [[(0, 0)], [(2, 1)], [(0, 1), (1, 1), (2, 1)]]
answer_text = [["Brad Pitt"], ["69"], ["209"]]
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(
    table=table,
    queries=queries,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)
inputs

{'input_ids': tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]]), 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'numeric_values': tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]]), 'numeric_values_scale': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [ ]:
table

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69


## TAPAS for Question Answering pretrained

Using Different methodology to defint the predefine the logits

In [ ]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

data = {
    "Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
    "Age": ["56", "45", "59"],
    "Number of movies": ["87", "53", "69"],
}
table = pd.DataFrame.from_dict(data)
queries = ["How many movies has George Clooney played in?", "How old is Brad Pitt?"]

inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)

logits = outputs.logits
logits_aggregation = outputs.logits_aggregation

### Taking the Inference of TAPAS Pretrained models using the cricketers data.
Conducted a small POC to check how accurately TAPAS model used the predefined modes and predict the modelling approach

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
data = pd.read_csv("/content/gdrive/MyDrive/graduate_rate.csv")
data = data.astype(str)

Mounted at /content/gdrive


In [4]:
data

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


In [4]:
data.dropna()

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


In [7]:
data_new = data[(data['Reporting_Level']=='School') & (data['Reporting_Label']=='ALL Students')]

In [14]:
data_new1 = data_new[data_new['Graduation_Rate']!='Too Few Students']

In [20]:
data_sorted = data_new1.sort_values(by='Graduation_Rate',ascending = False)
data_soted_new = data_sorted[data_sorted['Graduation_Rate']!='No Data']

In [30]:
 data_soted_new = data_soted_new.drop('Reporting_Level',axis=1)

In [23]:
data_soted_new.describe()

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
count,442,442,442,442,442,442,442
unique,1,188,200,188,433,1,360
top,School,667,0101,Gwinnett County,Lakeside High School,ALL Students,100.00
freq,442,26,14,26,2,442,18


In [27]:
data_soted_new[data_soted_new['Graduation_Rate']=='']

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate


In [5]:
# Defined the tapas model and the pipelinein which we are going to use it.
from transformers import pipeline
tqa = pipeline(task="table-question-answering", 
               model="google/tapas-base-finetuned-wtq")

In [37]:
data_soted_new.drop(['System_ID','School_ID'],axis=1,inplace = True)

In [51]:
data_soted_new = data_soted_new.reset_index(drop='True')

In [52]:
data_soted_new

,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,DeKalb County,Arabia Mountain High School - Academy of Engin...,ALL Students,99.67
1,Clayton County,Martha Ellen Stilwell School for the Performin...,ALL Students,99.28
2,Columbia County,Greenbrier High School,ALL Students,99.21
3,Atlanta Public Schools,Charles Drew Charter School JA/SA,ALL Students,99.10
4,Savannah-Chatham County,Savannah Arts Academy,ALL Students,99.10
...,...,...,...,...
437,Cobb County,Devereux Ackerman Academy,ALL Students,0.00
438,Savannah-Chatham County,UHS of Savannah Coastal Harbor Treatment Center,ALL Students,0.00
439,Taylor County,Georgia Center,ALL Students,0.00
440,Gwinnett County,Gwinnett Intervention Education Center (GIVE) ...,ALL Students,0.00


In [43]:
all_new_data = pd.DataFrame(data_soted_new.to_dict('records'))

In [47]:
all_new_data.rename(column = {'System_Name':'system name',
                              'School_Name':'school name',
                              'Reporting_Label':'reporting',
                              'Graduation_Rate':'graduation'},inplace= True)

TypeError: ignored

In [57]:
pos = range(1,443)
data_soted_new['Pos']=pos
data_soted_new.head()

,System_Name,School_Name,Reporting_Label,Graduation_Rate,Pos
0,DeKalb County,Arabia Mountain High School - Academy of Engin...,ALL Students,99.67,1
1,Clayton County,Martha Ellen Stilwell School for the Performin...,ALL Students,99.28,2
2,Columbia County,Greenbrier High School,ALL Students,99.21,3
3,Atlanta Public Schools,Charles Drew Charter School JA/SA,ALL Students,99.10,4
4,Savannah-Chatham County,Savannah Arts Academy,ALL Students,99.10,5


In [61]:
data_soted_new['Pos'] = data_soted_new['Pos'].astype('str')

In [62]:
qureies = ["what is the School_Name with highest Graduation_Rate"]
answer_list = tqa(table=data_soted_new,
                  query=qureies,
                  )

IndexError: ignored

In [33]:
answer_list

{'answer': 'System',
 'coordinates': [(13, 0)],
 'cells': ['System'],
 'aggregator': 'NONE'}

In [ ]:
for answer in answer_list:
    print(answer["answer"])

1992-2007
Sachin Tendulkar
SUM > 18426, 14234, 13704, 13430, 12650, 11867, 11739, 11579, 11363, 10889
Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid


In [ ]:
answer

{'answer': 'Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid',
 'coordinates': [(0, 1), (5, 1), (8, 1), (9, 1)],
 'cells': ['Sachin Tendulkar',
  'Virat Kohli',
  'Saurav Ganguly',
  'Rahul Dravid'],
 'aggregator': 'NONE'}